### All the necessary Imports

In [28]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Defining League and Seasons to consider

In [17]:
league_key = {'England':1,'Spain':31,'Germany':32,'Italy':33,'France':34,'Holland':36,'Portugal':37,'Scotland':10,'Turkey':79,'Championship':2}

In [18]:
season_key = {2019:16,2018:15,2017:14,2016:13,2015:12,2014:11,2013:10,2012:9,2011:8,2010:7,2009:6,2008:5,2007:4,2006:3}

### Defining a function to split one list into a list of smaller lists

In [19]:
def divide_chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n]

### Defining a function to get league data based on input League and Season

In [20]:
def get_league_data(league,season):
    #Building the URL
    data_url = 'https://thefishy.co.uk/leaguetable.php?table='+str(league_key[league])+'&season='+str(season_key[season])
    
    #Use Beautiful Soup to pull all contents of the web-page
    page = requests.get(data_url)
    soup = BeautifulSoup(page.content,'html.parser')
    
    #Find all tables in the page
    tables = soup.findAll("table")
    
    #Selecting the correct table to pull data from
    table_to_use = tables[5]
    
    #Extracting the header details
    table_header = table_to_use.find_all('th')
    header_list = []
    for i in table_header:
        header_list.append(i.contents[0])
        
    #Extracting table contents and cleaning data
    table_data = table_to_use.find_all('td')
    
    #Cleaning the table by removing tags
    table_data_clean = []
    for i in table_data:
        table_data_clean.append(i.contents[0])
            
    #Converting the larger list into a list of lists
    n = 10  
    x = list(divide_chunks(table_data_clean, n))
    
    #Converting the list of lists into a dataframe
    df = pd.DataFrame(x, columns =header_list)
    
    #Removing the tag values from the Team column
    df_team = df['Team']
    for i in range(0,len(df_team)):
        df_team[i] = df_team[i].contents[0]
    df['Team'] = df_team
    return df

In [42]:
get_league_data('Turkey',2017)

,Pos,Team,P,W,D,L,F,A,GD,Pts


In [36]:
get_league_data('France',2006).to_csv('France_Data_Csv',index=False)